In [1]:
import pandas as pd
import numpy as np
import nltk


In [2]:
#conda install -c conda-forge imbalanced-learn

In [2]:
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [6]:
from sklearn.preprocessing import LabelBinarizer

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [8]:
from collections import Counter
from imblearn.over_sampling import SMOTE

In [9]:
import tensorflow as tf
from keras.models import Sequential

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Input
from keras.optimizers import Adam, SGD

In [11]:
data = pd.read_csv('all-data.csv', names=['sentiment','text'], encoding='latin-1',header=None) 

In [12]:
data.head()

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [13]:
data.describe() # there are total 4846 reviews 

,sentiment,text
count,4846,4846
unique,3,4838
top,neutral,The company serves customers in various indust...
freq,2879,2


In [14]:
data['sentiment'].value_counts()

neutral     2879
positive    1363
negative     604
Name: sentiment, dtype: int64

In [15]:
len(data[data.duplicated()])

6

In [16]:
data=data.drop_duplicates()

In [17]:
print(data.shape)

(4840, 2)


In [19]:
#!pip install nltk

In [20]:
#nltk.download()

In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raehy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raehy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [20]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [21]:
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [22]:
def remove_special_characters(text, remove_digits=True):
  pattern=r'[^a-zA-z0-9\s]'
  text=re.sub(pattern,'',text) 
  return text

In [23]:
def delete_number(text, remove_digits=True):
    text= re.sub("\d+", "", s)
    return text 

In [24]:
data['text']=data['text'].apply(remove_special_characters)

In [25]:
# Lable target variable values in numeric type
label_mapping = {
        'negative': 0,
        'neutral': 1,
        'positive': 2
    }
    


In [26]:
data['sentiment']=data['sentiment'].replace(label_mapping) 

In [27]:
data.head()

,sentiment,text
0,1,According to Gran the company has no plans to...
1,1,Technopolis plans to develop in stages an area...
2,0,The international electronic industry company ...
3,2,With the new production plant the company woul...
4,2,According to the company s updated strategy fo...


### Removing Stopwords

In [28]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')
#set stopwords to english

print(stopword_list)


#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
data['text']=data['text'].apply(remove_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Normalized train ['text'] 

In [29]:
#normalized train text
norm_train_text=data.text[:4000]
norm_train_text[1]

'Technopolis plans develop stages area less 100000 square meters order host companies working computer technologies telecommunications statement said'

### Normalized test ['text']

In [30]:
#Normalized test reviews
norm_test_text=data.text[4000:]
norm_test_text[4845]

'Sales Finland decreased 105 January sales outside Finland dropped 17'

### Bags of words model  

In [31]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_text=cv.fit_transform(norm_train_text)
#transformed test reviews
cv_test_text=cv.transform(norm_test_text)


In [32]:
print('BOW_cv_train:',cv_train_text.shape)
print('BOW_cv_test:',cv_test_text.shape)

BOW_cv_train: (4000, 76187)
BOW_cv_test: (840, 76187)


In [33]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_text=tv.fit_transform(norm_train_text)
#transformed test reviews
tv_test_text=tv.transform(norm_test_text)

In [34]:
print('Tfidf_train:',tv_train_text.shape)
print('Tfidf_test:',tv_test_text.shape)

Tfidf_train: (4000, 76187)
Tfidf_test: (840, 76187)


## Data Modeling 

### Logistic Regression

In [35]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_text,data.sentiment[:4000])
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_text,data.sentiment[:4000])
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


### Training Dataset

In [36]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_text)

##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_text)


### Accuracy of the Model

In [37]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(data.sentiment[4000:],lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(data.sentiment[4000:],lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_bow_score : 0.34404761904761905
lr_tfidf_score : 0.3464285714285714


### Classification Report

In [38]:
#Classification report for bag of words 
lr_bow_report=classification_report(data.sentiment[4000:],lr_bow_predict,target_names=['positive','neutral','negative'])
print(lr_bow_report)
#Classification report for tfidf features
lr_tfidf_report=classification_report(data.sentiment[4000:],lr_tfidf_predict,target_names=['positive','neutral','negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       474
     neutral       0.37      0.97      0.53       290
    negative       0.10      0.09      0.09        76

    accuracy                           0.34       840
   macro avg       0.15      0.35      0.21       840
weighted avg       0.14      0.34      0.19       840

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       474
     neutral       0.35      1.00      0.52       290
    negative       0.14      0.01      0.02        76

    accuracy                           0.35       840
   macro avg       0.16      0.34      0.18       840
weighted avg       0.13      0.35      0.18       840



C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raehy\.

### Naive Beyes 

In [39]:
#training the model
mnb=MultinomialNB()
#Fitting the model for Bag of words
mnb_bow=mnb.fit(cv_train_text,data.sentiment[:4000])
print(mnb_bow)
#Fitting the model for tfidf features
mnb_tfidf=mnb.fit(tv_train_text,data.sentiment[:4000])
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


### Testing dataset

In [40]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_text)

##Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_text)

### Accuracy of the Model 

In [41]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(data.sentiment[4000:],mnb_bow_predict)
print("lr_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(data.sentiment[4000:],mnb_tfidf_predict)
print("lr_tfidf_score :",mnb_tfidf_score)

lr_bow_score : 0.34523809523809523
lr_tfidf_score : 0.34523809523809523


In [97]:
### Very low accuracy rate, I will apply additional method to improve the accuracy rate

### Alternative method (Spliting, Vectorizer and Modeling)

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
import pickle



In [43]:
X=data['text']
Y=data['sentiment']
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.3)

In [44]:
pipeline = Pipeline ([('vect', tv),('chi', SelectKBest(chi2, k=1200)), ('clf',RandomForestClassifier())])

In [45]:
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, Y_train)
with open('RandomForest.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(Y_test)

In [51]:
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.43      0.02      0.03       183
           1       0.59      1.00      0.74       855
           2       0.00      0.00      0.00       414

    accuracy                           0.59      1452
   macro avg       0.34      0.34      0.26      1452
weighted avg       0.40      0.59      0.44      1452

[[  3 180   0]
 [  3 852   0]
 [  1 413   0]]


## Fixing Imbalanced Target Variable 

#### The three classes - postiive, neutral and negative are not balanced that disrupts accurate prediction 

In [48]:
smote = SMOTE(random_state=777,k_neighbors=5)
X_smote,y_smote = smote.fit_sample(X_train,Y_train)

AttributeError: 'SMOTE' object has no attribute 'fit_sample'

In [47]:
from sklearn.linear_model import SGDClassifier

In [46]:
X_train, x_test, Y_train, y_test = train_test_split(X_smote,y_smote,test_size=0.3,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)


NameError: name 'X_smote' is not defined

In [ ]:
pred_sgd = sgd.predict(x_test)
print(accuracy_score(pred_sgd,y_test))
print(classification_report(y_test,pred_sgd))

## Neural Network 

### Tokenization  (first attempt)

In [97]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Dropout

In [98]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
tokenized_train = tok.texts_to_sequences(X_train)


In [99]:
X_train = pad_sequences(tokenized_train, maxlen=max_len)

### Data Modeling

In [83]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu')) 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [84]:
batch_size =32
epochs =10
history = model.fit(X,Y,batch_size, epochs, verbose=0)

ValueError: in user code:

    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\l

### Tokenization (second attempt)

In [49]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [51]:
num_classes = max(Y_train) + 1

In [52]:
max_words = 1000

In [54]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


In [55]:
vocab_size= len(tokenizer.word_index)+1

In [58]:
maxlen=200

In [60]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu')) 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size =32
epochs =10
history = model.fit(X,Y,batch_size, epochs, verbose=0)

### Tokenization (third attempt)

In [66]:
tokenizer= Tokenizer(num_words=1500, split=' ')

In [67]:
tokenizer.fit_on_texts(data['text'].values)

In [68]:
X=tokenizer.texts_to_sequences(data['text'])

In [69]:
X=pad_sequences(X)

### Data Modeling (LSTM)

In [96]:
from keras import regularizers

In [85]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length =2))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout=0.2))

model.add(Dense(2,activation='relu'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 128)            320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402       
Total params: 583,602
Trainable params: 583,602
Non-trainable params: 0
_________________________________________________________________
None


In [87]:
#### Encoding Y, the target variable

In [88]:
from sklearn.preprocessing import LabelEncoder

In [89]:
Le=LabelEncoder()
Y=Le.fit_transform(data['sentiment'])
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3, random_state =42)

In [90]:
model.fit(X_train,Y_train, epochs=10,batch_size=32)

Epoch 1/10


ValueError: in user code:

    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\raehy\.conda\envs\tensorflow\l

#### Regularization - Early Dropping 

In [92]:
from keras.callbacks import EarlyStopping

In [ ]:
model.fit(X_train, Y_train, epochs=100, callbacks = [EarlyStopping(monitor='acc', patience=3)])

y_train_pred = model.predict_classes(X_train)
y_test_pred = model.predict_classes(X_test)

print("The Accuracy score on the Train set is:\t{:0.3f}".format(accuracy_score(y_train, y_train_pred)))
print("The Accuracy score on the Test set is:\t{:0.3f}".format(accuracy_score(y_test, y_test_pred)))